<a href="https://colab.research.google.com/github/nekrut/BMMB554/blob/master/2021/ipynb/Pandas1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Pandas 1 | Basics

-------


> This is a reworked version of tutorial by [Justin Bois](http://justinbois.github.io/bootcamp/2020/index.html). It contains modified training datasets and adopts content to Colab environment. 

In [ ]:
import numpy as np

# Pandas, conventionally imported as pd
import pandas as pd

Throughout your research career, you will undoubtedly need to handle data, possibly lots of data. The data comes in lots of formats, and you will spend much of your time **wrangling** the data to get it into a usable form.

Pandas is the primary tool in the Python ecosystem for handling data. Its primary object, the `DataFrame` is extremely useful in wrangling data. We will explore some of that functionality here, and will put it to use in the next lesson.

## The data set

The dataset we will be using is a list of all SARS-CoV-2 datasets in [Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra) as of January 20, 2021.  

It is obtained by going to https://www.ncbi.nlm.nih.gov/sra and performing a query with the following search terms: `txid2697049[Organism:noexp]`.

Results are downloaded using `Send to:` menu selecting `File` and then `RunInfo`. Let's get these results into this notebook:

In [ ]:
!wget https://github.com/nekrut/BMMB554/raw/master/2021/data/sra_ncov_bmmb554.csv.gz

In [ ]:
!gunzip -c  sra_ncov_bmmb554.csv.gz | head

The first line contains the **headers** for each column. The data follow. Given the file I/O skills you recently learned, you could write some functions to parse this file and extract the data you want. You can imagine that this might be kind of painful. However, if the file format is nice and clean, like we more or less have here, we can use pre-built tools. Pandas has a very powerful function, `pd.read_csv()` that can read in a CSV file and store the contents in a convenient data structure called a **data frame**. In Pandas, the data type for a data frame is `DataFrame`, and we will use "data frame" and "`DataFrame`" interchangeably.

## Reading in data

Let's first look at the doc string of `pd.read_csv()`.

In [ ]:
pd.read_csv?

Holy cow! There are so many options we can specify for reading in a CSV file. You will likely find reasons to use many of these throughout your research. For now, however, we do not need most of them. So, let's load in the data set. Note that despite the fact that the datset is comprssed with `gzip` we do not need to do anything additional - pandas magically understands and uncompresses the data while loading it into the dataframe. 

In [ ]:
df = pd.read_csv('sra_ncov_bmmb554.csv.gz')

# Check the type
type(df)

We now have the data stored in a data frame. We can look at it in the Jupyter notebook, since Jupyter will display it in a well-organized, pretty way. Note that because our dataframe is big, we only display first five rows using `head()` function:

In [ ]:
df.head()

## Indexing data frames

The data frame is a convenient data structure for many reasons that will become clear as we start exploring. Let's start by looking at how data frames are indexed. Let's try to look at the first row.

In [ ]:
df[0]

Yikes! Lots of errors. The problem is that we tried to index numerically by row. **We index DataFrames, by columns.** And there is no column that has the name `0` in this data frame, though there could be. Instead, a might want to look at the column with the percentage of correct face matching tasks.

In [ ]:
df['Run'].head()

This gave us the numbers we were after. Notice that when it was printed, the index of the rows came along with it. If we wanted to pull out a single percentage correct, say corresponding to index `4`, we can do that.

In [ ]:
df['Run'][4]

However, this is **not** the preferred way to do this. It is better to use `.loc`. This give the location in the data frame we want.

In [ ]:
df.loc[4, 'Run']

It is also important to note that **row indices need not be integers**. And you should not count on them being integers. In practice you will almost never use row indices, but rather use **Boolean indexing**.

## Boolean indexing of data frames

Let's say I wanted to pull out accession numbers of runs produced by Pacfic Biosciences machines (in this table such datasets are labeled as `PACBIO_SMRT`. I can use Boolean indexing to specify the row. Specifically, I want the row for which `df['Platform'] == 'PACBIO_SMRT'`. You can essentially plop this syntax directly when using `.loc`.

In [ ]:
df.loc[df['Platform'] == 'PACBIO_SMRT', 'Run']

If I want to pull the whole record for that participant, I can use `:` for the column index.

In [ ]:
df.loc[df['Platform'] == 'PACBIO_SMRT', :]

Notice that indexes (the first column), come along for the ride, but we do not need them.

Now, let's pull out all PacBio records that were obtained from Amplicon sequencing. We can again use Boolean indexing, but we need to use an `&` operator. We did not cover this bitwise operator before, but the syntax is self-explanatory in the example below. Note that it is important that each Boolean operation you are doing is in parentheses because of the precedence of the operators involved.

In [ ]:
df.loc[(df['Platform'] == 'PACBIO_SMRT') & (df['LibraryStrategy'] == 'AMPLICON'), :]

What is going on will be clearer if we set up our Boolean indexing first, as follows.

In [ ]:
inds = (df['Platform'] == 'PACBIO_SMRT') & (df['LibraryStrategy'] == 'AMPLICON')

# Take a look
inds

Notice that `inds` is an array (actually a Pandas `Series`, essentially a `DataFrame` with one column) of `True`s and `False`s. We can apply `unique` function from [NumPy](https://numpy.org/) (we will talk about numpy next week) to see how many `True` and `False` rows we have:

In [ ]:
import numpy as np
np.unique(inds, return_counts=True)

When we index with it using .loc, we get back rows where inds is True.

In [ ]:
df.loc[inds, :]

Of interest in this exercise in Boolean indexing is that we never had to write a loop. To produce our indices, we could have done the following.

In [ ]:
# Initialize array of Boolean indices
inds = [False] * len(df)

# Iterate over the rows of the DataFrame to check if the row should be included
for i, r in df.iterrows():
    if r['Platform'] == 'PACBIO_SMRT' and r['LibraryStrategy'] == 'AMPLICON':
        inds[i] = True

# Make our seleciton with Boolean indexing
df.loc[inds, :]

This feature, where the looping is done automatically on Pandas objects like data frames, is very powerful and saves us writing lots of lines of code. This example also showed how to use the `iterrows()` method of a data frame to iterate over the rows of a data frame. It is actually rare that you will need to do that, as we'll show next when computing with data frames.

## Calculating with data frames

The SRA data contains (sometimes) size of the run in MB (`size_MB`). Let's suppose we want to consider only those run where `size_MB` is above 100. We might like to add a column to the data frame that specifies whether or not the corresponding run is above 100Mb. We can conveniently compute with columns. This is done elementwise.

In [ ]:
df['size_MB'] >= 100

This tells us which run is abobe 100 Mb. We can simply add this back to the data frame.

In [ ]:
# Add the column to the DataFrame
df['Over100Mb'] = df['size_MB'] >= 100

# Take a look
df.head()

## A note about vectorization

Notice how applying the `<=` operator to a `Series` resulted in **elementwise** application. This is called `vectorization`. It means that we do not have to write a `for` loop to do operations on the elements of a `Series` or other array-like object. Imagine if we had to do that with a `for` loop.

In [ ]:
big_runs = []
for size in df['size_MB']:
    big_runs.append(size >= 100)

This is cumbersome. The vectorization allows for much more convenient calculation. Beyond that, the vectorized code is almost always faster when using Pandas and Numpy because the looping is done with compiled code under the hood. This can be done with many operators, including those you've already seen, like `+`, `-`, `*`, `/`, `**`, etc.

## Applying functions to Pandas objects

Remember when we briefly saw the `np.mean()` function? We can compute with that as well. Let's compare the mean run size for Ilumina versus everything else.

In [ ]:
print('Illumina:          ', np.mean(df.loc[df['Over100Mb'], 'size_MB']))
print('Other platforms:   ', np.mean(df.loc[~df['Over100Mb'], 'size_MB']))

Notice that I used the `~` operator, which is a bit switcher. It changes all `True`s to `False`s and vice versa. In this case, it functions like NOT.

We will do a lot more computing with Pandas data frames in the next lessons. For our last demonstration in this lesson, we can quickly compute summary statistics about each column of a data frame using its `describe()` method.

In [ ]:
df.describe()

This gives us a data frame with summary statistics. Note that in this data frame, the row indices are not integers, but are the names of the summary statistics. If we wanted to extract the median value of each entry, we could do that with `.loc`.

In [ ]:
df.describe().loc['50%', :]

### Outputting a new CSV file

Now that we added the `Over100Mb` column, we might like to save our data frame as a new CSV that we can reload later. We use `df.to_csv()` for this with the `index` kwarg to ask Pandas not to explicitly write the indices to the file.

In [ ]:
df.to_csv('over100Mb_data.csv', index=False)

Let's take a look at what this file looks like.

In [ ]:
!head over100Mb_data.csv

Very nice. Notice that by default Pandas leaves an empty field for NaNs, and we do not need the `na_values` kwarg when we load in this CSV file.

## Computing environment

In [ ]:
!pip install watermark

In [ ]:
%load_ext watermark
%watermark -v -p numpy,pandas